In [1]:
%load_ext autoreload

# TODO
Anchor - inny atom niż C5'
Positions - normalizacja koordynatów przestrzennych (robiona już w kodzie treningowym)
    - w jaki sposób?
Jak to zrobić?
* Atom c5' jest zawsze w punkcie (0,0,0) a pozostałe atomy są wokół niego - raczej nie
* Policzyć średnią pozycję atomów wokół c5' i przesunąć wszystkie atomy o tą wartość. Dla każdej cząsteczki (pary nt) z osobna liczy się średnią i przesuwa. Oczekiwane wartości to punkt (0,0,0) gdzieś w środku pomiędzy nukleotydami.

Zwiększyć batch_size lub liczbę epok

In [1]:
import numpy as np
from Bio.SVDSuperimposer import SVDSuperimposer

def superimpose(ref_coords, model_coords):
    sup = SVDSuperimposer()
    sup.set(np.array(ref_coords, 'f'), np.array(model_coords, 'f'))
    sup.run()
    print('RMSD: {:.2f}'.format(sup.get_rms()))
    return sup.get_transformed()

def get_structure(parser, path):
    return parser.get_structure("str", path)[0]



In [5]:
from rnapolis.annotator import extract_secondary_structure
from rnapolis.parser import read_3d_structure

pdb_path = "/data/3d/input_data/pdbs/2A43_1_A.pdb"

# read 3d structure
with open(pdb_path) as f:
    structure3d = read_3d_structure(f, 1)
    structure2d = extract_secondary_structure(structure3d, 1)

interest_res = []
interest_bps = []
for bp in structure2d.basePairs:
    bp_type = bp.saenger.is_canonical if bp.saenger is not None else False
    print(bp_type, bp.nt1, bp.nt2)
    if bp_type and bp.nt1.name == 'G' or bp.nt2.name == 'G':
        print(f"{bp.nt1.chain}.{bp.nt1.name}{bp.nt1.number}")
        print(f"{bp.nt2.chain}.{bp.nt2.name}{bp.nt2.number}")
        bp1 = f"{bp.nt1.chain}.{bp.nt1.name}{bp.nt1.number}"
        bp2 = f"{bp.nt2.chain}.{bp.nt2.name}{bp.nt2.number}"
        interest_res.append(bp1)
        interest_res.append(bp2)
        interest_bps.append((bp1, bp2))
    # if not bp_type:
    #     print(bp_type, bp.nt1, bp.nt2)
    #     print(f"{bp.nt1.chain}.{bp.nt1.name}{bp.nt1.number}")
    #     print(f"{bp.nt2.chain}.{bp.nt2.name}{bp.nt2.number}")
    #     bp1 = f"{bp.nt1.chain}.{bp.nt1.name}{bp.nt1.number}"
    #     bp2 = f"{bp.nt2.chain}.{bp.nt2.name}{bp.nt2.number}"
    #     interest_res.append(bp1)
    #     interest_res.append(bp2)
    #     interest_bps.append((bp1, bp2))

True Residue(label=None, auth=ResidueAuth(chain='A', number=3, icode=None, name='G')) Residue(label=None, auth=ResidueAuth(chain='A', number=16, icode=None, name='C'))
A.G3
A.C16
False Residue(label=None, auth=ResidueAuth(chain='A', number=3, icode=None, name='G')) Residue(label=None, auth=ResidueAuth(chain='A', number=20, icode=None, name='A'))
True Residue(label=None, auth=ResidueAuth(chain='A', number=4, icode=None, name='C')) Residue(label=None, auth=ResidueAuth(chain='A', number=15, icode=None, name='G'))
A.C4
A.G15
True Residue(label=None, auth=ResidueAuth(chain='A', number=5, icode=None, name='G')) Residue(label=None, auth=ResidueAuth(chain='A', number=14, icode=None, name='C'))
A.G5
A.C14
True Residue(label=None, auth=ResidueAuth(chain='A', number=6, icode=None, name='G')) Residue(label=None, auth=ResidueAuth(chain='A', number=13, icode=None, name='C'))
A.G6
A.C13
False Residue(label=None, auth=ResidueAuth(chain='A', number=6, icode=None, name='G')) Residue(label=None, auth=Res

In [6]:
from Bio.PDB import *

residues = {}
# Read the pdb file
parser = PDBParser()
structure = parser.get_structure('pdb', pdb_path)
for residue in structure.get_residues():
    # print(f"{residue.get_segid().strip()}.{residue.get_resname()}{residue.get_id()[1]}")
    # print(next(residue.get_atoms()).get_coord())
    res = f"{residue.get_full_id()[2]}.{residue.get_resname()}{residue.get_id()[1]}"
    
    if res in interest_res:
        print(res)
        residues[res] = residue

A.G3
A.C4
A.G5
A.G6
A.C7
A.C9
A.C10
A.G11
A.C13
A.C14
A.G15
A.C16
A.C26
A.G27
A.G28


/data/miniconda3/envs/difflinker/lib/python3.10/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 574
  warnings.warn(


In [8]:
def save_as_new_pdb(residues, output_file):
    structure = Structure.Structure('pdb')
    model = Model.Model(0)
    chain = Chain.Chain('A')
    model.add(chain)
    structure.add(model)

    # add the residues to the structure
    for residue in residues:
        chain.add(residue)

    # write the new pdb file
    io = PDBIO()
    io.set_structure(structure)
    io.save(output_file)

for i, bps in enumerate(interest_bps):
    print(bps)
    sel_res = [residues[bps[0]], residues[bps[1]]]
    save_as_new_pdb(sel_res, f"my_out{i}.pdb")

('A.G3', 'A.C16')
('A.C4', 'A.G15')
('A.G5', 'A.C14')
('A.G6', 'A.C13')
('A.C7', 'A.G11')
('A.C9', 'A.G28')
('A.C10', 'A.G27')
('A.G11', 'A.C26')


In [32]:
%autoreload 2
from data_processing import pdb_transform, get_one_hot

In [28]:
transformed = pdb_transform(sel_res, pdb_path)


In [49]:
# save transformed to *.pt file
import torch
name = 'rna_GC_val.pt'
# torch.save([transformed], name)

In [50]:
import torch
pt_data = torch.load(name)
print(len(pt_data))
pt_data[0].keys()

3


dict_keys(['uuid', 'name', 'positions', 'one_hot', 'fragment_mask', 'linker_mask', 'num_atoms', 'anchors', 'charges'])

In [51]:
import numpy as np
a = [-10, 0, 10]
b = [0, 10, 20]
a1 = a/np.max(a)
b1 = b/np.max(b)
print(a1)
print(b1)

# translate coordinates, so the a is in the origin and b is on the x-axis
a = np.array(a)
b = np.array(b)
a = a - np.mean(a)
b = b - np.mean(b)
print(a)
print(b)



[-1.  0.  1.]
[0.  0.5 1. ]
[-10.   0.  10.]
[-10.   0.  10.]


In [53]:
index = 0
pos = pt_data[index]['positions'].numpy() - np.mean(pt_data[index]['positions'].numpy(), axis=0)
pos

array([[-1.9320006 ,  8.634837  , -3.0085347 ],
       [-2.6279998 ,  6.793837  , -4.748535  ],
       [-2.3770003 ,  7.151837  , -3.380535  ],
       [-2.6090007 ,  5.294837  , -4.9125347 ],
       [-1.2849994 ,  4.780837  , -4.613535  ],
       [-3.5230002 ,  4.502837  , -3.9915347 ],
       [-4.8700004 ,  4.472837  , -4.427535  ],
       [-2.8819995 ,  3.124837  , -4.028535  ],
       [-3.2189994 ,  2.443837  , -5.224535  ],
       [-1.3980002 ,  3.487837  , -4.035535  ],
       [-0.8639999 ,  3.525837  , -2.6565347 ],
       [-0.43300012,  2.328837  , -2.0725348 ],
       [-0.51099926,  1.283837  , -2.7285347 ],
       [ 0.04999968,  2.3448372 , -0.80953485],
       [ 0.10199973,  3.489837  , -0.12853485],
       [ 0.58400005,  3.456837  ,  1.1184651 ],
       [-0.336     ,  4.722837  , -0.69253474],
       [-0.8019996 ,  4.697837  , -1.9485348 ],
       [-2.8730006 ,  9.559837  , -3.7035348 ],
       [-1.7839999 ,  8.712837  , -1.5345348 ],
       [ 3.776999  , -6.2121634 ,  5.989

In [54]:
np.mean(pt_data[index]['positions'].numpy(), axis=0)

array([-6.1656156e-06, -6.6535421e-08, -1.7742778e-07], dtype=float32)

In [48]:
pt_data[index]['positions'][3] - pt_data[index]['positions'][5]

tensor([-0.6830,  0.9330,  0.9930])

In [16]:
pt_data[5]['positions'][0] - pt_data[5]['positions'][1]

tensor([0.4591, 0.8352, 1.1577], device='cuda:0')

In [18]:
pt_data[5]['positions']

tensor([[ 2.0732e+00,  2.8905e+00,  1.1610e+00],
        [ 1.6141e+00,  2.0553e+00,  3.3000e-03],
        [ 2.7350e-01,  1.6952e+00, -1.2480e-01],
        [-1.4300e-01,  8.8410e-01, -1.1825e+00],
        [ 7.7240e-01,  4.1420e-01, -2.1436e+00],
        [ 2.1292e+00,  7.8020e-01, -2.0248e+00],
        [ 3.1977e+00,  3.0730e-01, -2.9754e+00],
        [ 2.5313e+00,  1.5722e+00, -9.3450e-01],
        [ 1.5150e-01, -9.0660e-01,  3.9207e+00],
        [-4.5100e-01,  6.5280e-01,  3.5543e+00],
        [-1.1879e+00,  5.1300e-02,  2.1060e+00],
        [-1.8739e+00,  8.2550e-01,  1.2206e+00],
        [-2.8154e+00,  2.0660e-01,  2.9710e-01],
        [-3.9177e+00,  1.2508e+00,  1.3900e-01],
        [-3.1829e+00,  2.5628e+00,  3.4380e-01],
        [-2.2021e+00,  2.2234e+00,  1.4632e+00],
        [-1.0060e+00, -1.2406e+00,  1.9257e+00],
        [-2.0120e-01, -1.7865e+00,  2.9222e+00],
        [ 2.1440e+00, -3.3391e+00, -1.0544e+00],
        [ 9.7650e-01, -3.0248e+00, -8.7670e-01],
        [ 1.4700e-01

In [55]:
index = 0
print(pt_data[index]['positions'].mean(), pt_data[index]['positions'].std())

tensor(-2.1365e-06) tensor(3.6918)


In [6]:
print(pt_data[0]['linker_mask'])
print(pt_data[0]['fragment_mask'])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1.])
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.])


In [4]:
for a in pt_data:
    print(a['name'])

1CSL_1_B-A_0
1CSL_1_B-A_1
1CSL_1_B-A_2
1CSL_1_B-A_3
1CSL_1_B-A_4
1CSL_1_B-A_5
1CSL_1_B-A_6
1D4R_1_B-A_0
1D4R_1_B-A_1
1D4R_1_B-A_2
1D4R_1_B-A_3
1D4R_1_B-A_4
1D4R_1_B-A_5
1D4R_1_B-A_6
1D4R_1_B-A_7
1D4R_1_B-A_8
1D4R_1_B-A_9
1D4R_1_B-A_10
1D4R_1_B-A_11
1D4R_1_B-A_12
1D4R_1_B-A_13
1D4R_1_B-A_14
1D4R_1_B-A_15
1D4R_1_B-A_16
1D4R_1_B-A_17
1DQF_1_B-A_0
1DQF_1_B-A_1
1DQF_1_B-A_2
1DQF_1_B-A_3
1DQF_1_B-A_4
1DQF_1_B-A_5
1DQF_1_B-A_6
1DQH_1_B-A_0
1DQH_1_B-A_1
1DQH_1_B-A_2
1DQH_1_B-A_3
1DQH_1_B-A_4
1DQH_1_B-A_5
1DQH_1_B-A_6
1DUQ_1_B-A_0
1DUQ_1_B-A_1
1DUQ_1_B-A_2
1DUQ_1_B-A_3
1DUQ_1_B-A_4
1DUQ_1_B-A_5
1DUQ_1_B-A_6
1DUQ_1_B-A_7
1DUQ_1_B-A_8
1ET4_1_A_0
1ET4_1_A_1
1ET4_1_A_2
1ET4_1_A_3
1ET4_1_A_4
1ET4_1_A_5
1ET4_1_A_6
1ET4_1_A_7
1ET4_1_A_8
1F1T_1_A_0
1F1T_1_A_1
1F1T_1_A_2
1F1T_1_A_3
1F1T_1_A_4
1F1T_1_A_5
1F1T_1_A_6
1F1T_1_A_7
1F1T_1_A_8
1F1T_1_A_9
1F1T_1_A_10
1F27_1_A_0
1F27_1_A_1
1F27_1_A_2
1FIR_1_A_0
1FIR_1_A_1
1FIR_1_A_2
1FIR_1_A_3
1FIR_1_A_4
1FIR_1_A_5
1FIR_1_A_6
1FIR_1_A_7
1FIR_1_A_8
1FIR_1_A_9
1FIR

In [ ]:
atoms = get_atoms(sel_res)
one_hot = get_one_hot(sel_res)
coding = {}
idx_atom = {}
simple_idx_atom = {}
for a, oh in zip(atoms, one_hot):
    print(a, np.argmax(oh))
    coding[a[0]] = np.argmax(oh)
    idx_atom[np.argmax(oh)] = a[0]
    simple_idx_atom[np.argmax(oh)] = a[0]

P 3
C5' 0
O5' 2
C4' 0
O4' 2
C3' 0
O3' 2
C2' 0
O2' 2
C1' 0
N1 1
C2 0
O2 2
N3 1
C4 0
O4 2
C5 0
C6 0
OP1 2
OP2 2
P 3
C5' 0
O5' 2
C4' 0
O4' 2
C3' 0
O3' 2
C2' 0
O2' 2
C1' 0
N1 1
C2 0
O2 2
N3 1
C4 0
O4 2
C5 0
C6 0
OP1 2
OP2 2


In [ ]:
simple_idx_atom

{3: 'P', 0: 'C', 2: 'O', 1: 'N'}

In [ ]:
coding

{'P': 3, 'C': 0, 'O': 2, 'N': 1}